<a href="https://colab.research.google.com/github/pranjul6386/covid19/blob/master/covid_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip "/content/drive/My Drive/kaggle_notebooks/mask.zip"

Archive:  /content/drive/My Drive/kaggle_notebooks/mask.zip
  inflating: annotations/maksssksksss0.xml  
  inflating: annotations/maksssksksss1.xml  
  inflating: annotations/maksssksksss10.xml  
  inflating: annotations/maksssksksss100.xml  
  inflating: annotations/maksssksksss101.xml  
  inflating: annotations/maksssksksss102.xml  
  inflating: annotations/maksssksksss103.xml  
  inflating: annotations/maksssksksss104.xml  
  inflating: annotations/maksssksksss105.xml  
  inflating: annotations/maksssksksss106.xml  
  inflating: annotations/maksssksksss107.xml  
  inflating: annotations/maksssksksss108.xml  
  inflating: annotations/maksssksksss109.xml  
  inflating: annotations/maksssksksss11.xml  
  inflating: annotations/maksssksksss110.xml  
  inflating: annotations/maksssksksss111.xml  
  inflating: annotations/maksssksksss112.xml  
  inflating: annotations/maksssksksss113.xml  
  inflating: annotations/maksssksksss114.xml  
  inflating: annotations/maksssksksss115.xml  
  infl

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import PIL
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
import xml.etree.ElementTree as ET 
import tqdm
import time
import datetime
import math

In [3]:

 
# read image
img = cv2.imread('/content/images/maksssksksss101.png', cv2.IMREAD_UNCHANGED)
 
# get dimensions of image
dimensions = img.shape

 
# height, width, number of channels in image
height = img.shape[0]
width = img.shape[1]
channels = img.shape[2]
 
print('Image Dimension    : ',dimensions)
print('Image Height       : ',height)
print('Image Width        : ',width)
print('Number of Channels : ',channels)

Image Dimension    :  (400, 301, 4)
Image Height       :  400
Image Width        :  301
Number of Channels :  4


In [4]:
count=0
for img in os.listdir("/content/images"):
  image=cv2.imread(img)
  count+=1
print(count)




853


In [12]:
sample_size=852
image_ann_dir="/content/annotations"
image_height=128
image_width=128
image_channels=3
image_input_dir="/content/images"

In [6]:
def read_image(src):
    img = cv2.imread(src)
    if img is None:
        raise FileNotFoundError
    img= cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img

In [13]:
def load_cropped_images(image_ann_dir=image_ann_dir, sample_size=sample_size, 
                        image_width=image_width, image_height=image_height, image_channels=image_channels,image_input_dir=image_input_dir):
    curIdx = 0
    labels = []
    images_np = np.zeros((sample_size,image_width,image_height,image_channels))       #initialize the feature vector by zeros
  
    for ann in os.listdir(image_ann_dir):
        for img in os.listdir(image_input_dir):
          try:
            image_path= str(image_input_dir + "/" + img)
            image=cv2.imread(image_path)
            image= cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)

  #get an image by reaching the folder 
          except FileNotFoundError:
            print("file not found")
            
            continue
           
        tree = ET.parse(os.path.join(image_ann_dir, ann))          #used XML parsing to form atree to find the bounding boxes
        root = tree.getroot()
        filename = root.find('filename').text
        size = root.find('size')
        width = int(size.find('width').text)
        height = int(size.find('height').text)
        objects = root.findall('object')
        for o in objects:
            bndbox = o.find('bndbox') 
            
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)          #dimensions of the bounding boxes 
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            xmin = int(max(0, xmin - 4))        # 2: margin
            xmax = int(min(width, xmax + 4))
            ymin = int(max(0, ymin - 4))
            ymax = int(min(height, ymax + 4))

            w = int(np.min((xmax - xmin, ymax - ymin)))
            w = int(min(w, width, height))
                              # available w

            if w > xmax - xmin:
                xmin = min(max(0, xmin - int((w - (xmax - xmin))/2)), width - w)
                xmax = xmin + w
            if w > ymax - ymin:
                ymin = min(max(0, ymin - int((w - (ymax - ymin))/2)), height - w)
                ymax = ymin + w
            
            img_cropped = image[ymin:ymin+w, xmin:xmin+w, :]      # [h,w,c]
            # Interpolation method
            if xmax - xmin > image_width:
                interpolation = cv2.INTER_AREA          # shrink
            else:
                interpolation = cv2.INTER_CUBIC 
            try:        # expansion
                
                img_cropped = cv2.resize(img_cropped, (image_width, image_height), 
                                      interpolation=interpolation)  # resize
            except:
              print("file not found")
              print(image_path)
              continue
                
            images_np[curIdx,:,:,:] = np.asarray(img_cropped)   #the finAL feature vector
            labels.append(filename)
            curIdx=curIdx+1
            
            
    return images_np,labels

In [14]:
start_time=time.time()
images_np, labels = load_cropped_images(sample_size=852)
est_time=round(time.time()-start_time)
print("Feature loading time: {}.".format(str(datetime.timedelta(seconds=est_time))))

file not found
/content/images/maksssksksss722.png


IndexError: ignored

In [11]:
images_np

NameError: ignored

In [ ]:
labels

[]